# Running Differential Gene Expression Analyses on RNA-sequencing datasets... On Google Colab!
Welcome! This notebook's purpose is to serve as a one-stop-shop to run some standard Differential Gene Expression Analyses (DGEA) using [DESeq2](https://bioconductor.org/packages/release/bioc/html/DESeq2.html) and [edgeR](https://bioconductor.org/packages/release/bioc/html/edgeR.html) **without downloading anything on your computer**! 

The intuition behind how this works is as follows:

1. This is a notebook. Notebooks are useful to document your data exploration and analyses and then share it with other people. 

2. You can write and save code directly into a notebook, but you can also run code that has been pre-written by yourself/someone else. This notebook uses both of these approaches, but the bulk of the code is pre-written in separate .R and .py files. 

3. The code in this notebook runs on Google's servers, not on your computer. Each time you launch this notebook, all the software you need to run it is installed **on Google's servers**.

In [ ]:
#@title Load pre-written code from Github{display-mode: "form"}
#@markdown This notebook uses both `R` and `Python` code, and the bulk of it is pre-written.
#@markdown Here we simply download that pre-written code on Google's servers so that we can run it in the notebook.

#@markdown If you're re-running this chunk of the notebook, you might get the following error message:

#@markdown `fatal: destination path 'jd-rnaseq-repo' already exists and is not an empty directory.`

#@markdown That's fine, you can ignore that warning message.
!git clone https://github.com/srcoulombe/jd-rnaseq-repo.git


In [1]:
#@title Import the `Python` code that we just downloaded from Github{display-mode: "form"}
#@markdown Here we simply `import` the pre-written `Python` files. 
from importlib.machinery import SourceFileLoader

result_comparison = SourceFileLoader("result_comparison", "Python_code/result_comparison.py").load_module()
gose = SourceFileLoader("gose", "Python_code/gene_ontology_statistical_enrichment.py").load_module()

In [2]:
#@title Load `rpy2` {display-mode: "form"}
#@markdown This notebook uses both `R` and `Python` code, and the [`rpy2`](https://pypi.org/project/rpy2/) library allows to intermix both languages.
#@markdown If you're re-running this chunk of the notebook, you might get the following error message:

#@markdown `The rpy2.ipython extension is already loaded. To reload it, use: %reload_ext rpy2.ipython`

#@markdown That's fine, you can ignore that warning message.

%load_ext rpy2.ipython
# https://stackoverflow.com/questions/54595285/how-to-use-r-with-google-colaboratory
import warnings
warnings.filterwarnings(action='once')

import rpy2
import rpy2.robjects as ro


C:\Users\Samy\miniconda3\lib\site-packages\rpy2\robjects\pandas2ri.py:17: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


In [3]:
#@title Import the `R` code that we just downloaded from Github{display-mode: "form"}
#@markdown This will download the required `R` packages (e.g. [DESeq2](https://bioconductor.org/packages/release/bioc/html/DESeq2.html) and [edgeR](https://bioconductor.org/packages/release/bioc/html/edgeR.html)) on Google's servers. This might take a while (so now's a good time to get that cup of coffee)...
%%R
source("R_code/all_code.R")

UsageError: Line magic function `%%R` not found.


In [ ]:
#@title Uploading the read-count matrix file {display-mode: "form"}
#@markdown Now we need to upload the read-count matrix file we'll be working with to Google's servers (in order to read into its contents)!

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
import os
#@title Choosing parameters
#@markdown #Choosing parameters
#@markdown Enter the name (including the extension) of the read-count matrix file (e.g. mymatrix.csv):
file_path = "/content/QIAseqUltraplexRNA_90846_relabelled_alphaordered.csv" #@param {type:"string"}
#@markdown ---
try:
  assert os.path.isfile(file_path)
except AssertionError as file_not_found:
  error_message = "\n".join([
    "[x] Couldn't find the file named:",
    file_path,
    "Are you sure you've written the right file path?",
    "Hint: you can navigate to the file's location with the Google Colab's `Files` explorer, and right-click on the correct file to copy its `path` and paste it here."
  ])
  raise FileNotFoundError(error_message) from file_not_found
else:
  print(f"[✓] File {file_path} has been found!")

#@markdown Choose the false discovery rate (FDR)
fdr = 0.1 #@param {type:"slider", min:0.0, max:1.0, step:0.01}

#@markdown Choose the statistic to use for independent filtering
chosen_filter = 'mean' #@param ["mean", "min", "max", "median"]


#@markdown Enter the file name prefix you want to use to identify your output files
prefix = "atest" #@param {type:"string"}

#@markdown ---
try:
  files_with_prefix = [
    pre_existing_file for pre_existing_file 
    in os.listdir() if prefix in pre_existing_file                     
  ]
  assert files_with_prefix == []
except AssertionError as prefix_conflict:
  error_message = "\n".join([
    f"[x] Files with the prefix {prefix} already exist:",
    "\t"+'\n\t'.join(files_with_prefix),
    "Please choose another prefix."
  ])
  raise FileExistsError(error_message) from prefix_conflict
else:
  print(f"[✓] Prefix {prefix} is a valid prefix!")

# adding variables to R session
ro.r('counts.matrix.filepath="'+file_path+'"')
ro.r('fdr='+f"{fdr}")
ro.r('chosen_filter="'+chosen_filter+'"')
ro.r('prefix="'+prefix+'"')


In [ ]:
from pprint import pprint, pformat
import pandas as pd
df = pd.read_csv(file_path)
#@markdown Choose the columns to keep
keep_columns = [ 'RA_RepB', 'RA_RepC', 'siGFP_RepA', 'siGFP_RepB', 'siGFP_RepC', 'siM1_RepA', 'siM1_RepB', 'siM1_RepC', 'siNC_RepA', 'siNC_RepB', 'siNC_RepC'] #@param {type:"raw"}
for column_name in keep_columns:
  try:
    assert column_name in df.columns
  except AssertionError as column_not_found:
    error_message = "\n".join([
      f"The specified column: {column_name}",
      "was not found in the input file's columns.",
      "The valid column names are:",
      pformat(df.columns)
    ])
    raise ValueError(error_message) from column_not_found
  else:
    print(f"[✓] Found the {column_name} column")

formatted_keep_columns = ",".join([
  f'"{column}"' for column in keep_columns
])
print(formatted_keep_columns)
ro.r('keep_columns=c('+formatted_keep_columns+')')

In [ ]:
#@markdown #Reading the read-count matrix and visualizing the retained samples' read count distributions
%%R
raw.counts.data <- rawCountsMatrix_to_dataframe(
  counts.matrix.filepath,
  keep_columns=keep_columns,
  make_histogram=TRUE, make_boxplot=TRUE,
  make_ensembl_to_symbol = FALSE,
  sep = ',',
  verbose=TRUE
)

sample.data <- data.frame(
  condition=raw.counts.data$conditions,
  row.names=colnames(raw.counts.data$raw.data)
)

colnames(sample.data) <- c("condition")

contrasts <- design.pairs(unique(sample.data$condition))

# DGEA with DESeq2


In [ ]:
#@markdown ##Formatting the raw read-count matrix into the format that DESeq2 uses

%%R
dds <- DESeqDataSetFromMatrix(
  countData = raw.counts.data$raw.data,
  colData = sample.data,
  design = ~ condition
)

In [ ]:
#@markdown ##Plotting sample-wise correlation matrix (using DESeq2-normalized data)
%%R
plotPheatMap(dds, fromtool="deseq")

In [ ]:
#@markdown ##Plotting the Scree and PC1-vs-PC2 plots to compare samples based on their DESeq2-normalized read counts
%%R
rld <- rlog(dds, blind=TRUE)
prcomp.output <- prcomp(t(assay(rld)))
plotScree(prcomp.output)
plotPC1vsPC2(prcomp.output, dds)

In [ ]:
#@markdown ##Run DGEA with DESeq2 and plot the rejection curves obtained with DESeq2's independent filtering procedure
%%R
DESeq2.results <- DESeq2_DGE_analysis( 
    dds, 
    fdr, 
    contrasts, 
    verbose = FALSE, 
    chosen_filter = chosen_filter
)

In [ ]:
#@markdown ##Plotting the Scree and PC1-vs-PC2 plots to compare samples based on their DESeq2-normalized read counts that have passed the independent filtering step
%%R
DESeq2.normalized.data <- as.data.frame(counts(DESeq2.results$DGE_obj, normalized=TRUE))
prcomp.output <- prcomp(t(DESeq2.normalized.data))
plotScree(prcomp.output)
plotPC1vsPC2(prcomp.output, dds)

# DGEA with edgeR


In [ ]:
#@markdown ##Formatting the raw read-count matrix into the format that edgeR uses
%%R
dge_obj <- DGEList(
  counts=raw.counts.data$raw.data,
  group=raw.counts.data$conditions
)
design <- model.matrix(~0+sample.data$condition)
rownames(design) <- row.names(sample.data)
colnames(design) <- gsub("sample.data.condition","",colnames(design))

### We now run DGEA with edgeR, and plot the rejection curves obtained with independent filtering method



In [ ]:
#@markdown ##Run DGEA with edgeR and plot the rejection curves obtained with DESeq2's independent filtering procedure
%%R
edgeR.results <- edgeR_DGE_analysis(
    dge_obj, 
    design, 
    fdr, 
    contrasts = contrasts, 
    verbose = FALSE, 
    chosen_filter = chosen_filter
)

In [ ]:
#@markdown ##Plotting the Scree and PC1-vs-PC2 plots to compare samples based on their edgeR-normalized read counts that have passed the independent filtering step
%%R
prcomp.output <- prcomp(t(
   t(t(edgeR.results$DGE_obj$pseudo.counts)*(edgeR.results$DGE_obj$samples$norm.factors))
))
plotScree(prcomp.output)
plotPC1vsPC2(prcomp.output, dds)

## Now we save the results of our DGEA on your Google Drive (for bookkeeping and for downstream analyses)

In [ ]:
#@markdown ## First we want to find the gene symbol that corresponds to each Ensembl ID (since those are the gene identifiers we've been using so far). So the first step is to get that map from Ensembl.
%%R
symbol.to.id.map <- get.conversion.map(
    raw.counts.data,
    prefetched_file_name = "/content/jd-rnaseq-repo/R_code/ensembl_ID_gene_symbol_map_may2020.tsv"
)
print("Preview:")
head(symbol.to.id.map)

In [ ]:
#@markdown ## Finally, we save our DGEA results in a tab-separated file
%%R
spreadsheet_dir_path <- save.spreadsheet( 
  DESeq2.results, 
  edgeR.results, 
  raw.counts.data, 
  contrasts, 
  symbol.to.id.map, 
  fdr,
  prefix, 
  getwd(), 
  prefix,
  saveoutput = TRUE
)

In [ ]:
# Additional pathway analysis stuff would go here

## Post-DGEA visualizations

In [ ]:
#@markdown ## Compare the genes qualified as "differentially expressed" by the two tools at different FDRs
df_dict = result_comparison.load_files_from_dir(
    #"/content/drive/My Drive/msc-lncrna-project/jd-rnaseq/secondary_analysis/All_RAi_except_RAsA"
    str(rpy2.robjects.r['spreadsheet_dir_path'])[5:-2]
)
result_comparison.compare_with_max_adjp(df_dict, [0.05, 0.1])